In [1]:
!which pip3

/home/mgross/anaconda3/envs/contrast-net/bin/pip3


In [2]:
! nvidia-smi

Thu Jan 29 06:15:52 2026       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 545.23.06              Driver Version: 545.23.06    CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Quadro RTX 8000                Off | 00000000:17:00.0 Off |                  Off |
| 33%   29C    P8              23W / 260W |   3136MiB / 49152MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [3]:
import IPython
import matplotlib.pyplot as plt
import nibabel as nib
import numpy as np
import os
import pandas as pd
import pytorch_lightning
from monai.visualize import img2tensorboard
import torch
import time
import sklearn

from torch.utils.data import DataLoader

import monai
from monai.transforms import \
    Compose, LoadImaged, AddChanneld, Orientationd, \
    NormalizeIntensityd, ScaleIntensityRangePercentilesd, \
    SpatialPadd, Spacingd, Resized, \
    RandCropByPosNegLabeld, ToTensord,  \
    DataStatsd, AsDiscrete, KeepLargestConnectedComponent, RandFlipd
#LoadNiftid


from monai.data import list_data_collate
from monai.networks.layers import Norm

# from monai.metrics import compute_meandice
from monai.transforms import KeepLargestConnectedComponent

from monai.metrics import DiceMetric
from monai.inferers import sliding_window_inference
from monai.utils import set_determinism
from monai.metrics import compute_meandice
from monai.metrics import compute_hausdorff_distance
from monai.metrics import compute_average_surface_distance


from monai.data import write_nifti
from tqdm.notebook import tqdm





monai.config.print_config()

MONAI version: 0.7.0
Numpy version: 1.20.3
Pytorch version: 1.8.2
MONAI flags: HAS_EXT = False, USE_COMPILED = False
MONAI rev id: bfa054b9c3064628a21f4c35bbe3132964e91f43

Optional dependencies:
Pytorch Ignite version: NOT INSTALLED or UNKNOWN VERSION.
Nibabel version: 3.2.1
scikit-image version: 0.18.3
Pillow version: 8.3.1
Tensorboard version: 2.6.0
gdown version: NOT INSTALLED or UNKNOWN VERSION.
TorchVision version: 0.9.2
tqdm version: 4.62.3
lmdb version: NOT INSTALLED or UNKNOWN VERSION.
psutil version: NOT INSTALLED or UNKNOWN VERSION.
pandas version: 1.3.4
einops version: NOT INSTALLED or UNKNOWN VERSION.
transformers version: NOT INSTALLED or UNKNOWN VERSION.

For details about installing the optional dependencies, please visit:
    https://docs.monai.io/en/latest/installation.html#installing-the-recommended-dependencies



In [4]:
print('CUDA available: ', torch.cuda.is_available())

n_gpus = torch.cuda.device_count()
for i in range(n_gpus):
    print('GPU %d: %s' % (i, torch.cuda.get_device_name(i)))


CUDA available:  True
GPU 0: Quadro RTX 8000


## Load the Data

In [5]:
## Load the external data from Paris

import glob 
import pandas as pd

external_testing_img_paths = pd.read_csv('/tensor5/data1/jonofrey/Liver/Paris/registration/nonrigid_nmi/Nonrigid_Registered_NMI_Image_Paths.csv')

included_accs = [1,3, 4, 6, 10, 11, 12, 17, 18, 19, 22, 32, 35, 40, 55, 59]
external_testing_img_paths = external_testing_img_paths[external_testing_img_paths['ACCESSION_ID'].isin(included_accs)]

segm_path = '/tensor5/data1/mgross/Liver/Paris/man_liver_segms/*_PV_LIVER-label.nii.gz'
segm_paths = glob.glob(segm_path)

external_testing_accs = [int(i.split('/')[-1].split('_')[0]) for i in segm_paths]

df_external_testing = pd.DataFrame({'ACCESSION_ID': external_testing_accs, 'SEGM' : segm_paths})
df_external_testing = pd.merge(external_testing_img_paths, df_external_testing, on = 'ACCESSION_ID')

df_external_testing = df_external_testing[['ACCESSION_ID','PV_PRE','PV_ART','PV','PV_DEL',  'SEGM']]


df_gt = df_external_testing.copy()
df_gt.rename(columns={'SEGM': 'GT-SEGM'}, inplace = True)

for i in [df_gt]:
    for idx, row in  i.iterrows():
            i.loc[idx, 'ACCESSION_ID'] = str(row['ACCESSION_ID'])


df_gt


,ACCESSION_ID,PV_PRE,PV_ART,PV,PV_DEL,GT-SEGM
0,1,/tensor5/data1/jonofrey/Liver/Paris/registrati...,/tensor5/data1/jonofrey/Liver/Paris/registrati...,/tensor5/data1/mgross/Liver/Paris/00001/00001_...,/tensor5/data1/jonofrey/Liver/Paris/registrati...,/tensor5/data1/mgross/Liver/Paris/man_liver_se...
1,3,/tensor5/data1/jonofrey/Liver/Paris/registrati...,/tensor5/data1/jonofrey/Liver/Paris/registrati...,/tensor5/data1/mgross/Liver/Paris/00003/00003_...,/tensor5/data1/jonofrey/Liver/Paris/registrati...,/tensor5/data1/mgross/Liver/Paris/man_liver_se...
2,4,/tensor5/data1/jonofrey/Liver/Paris/registrati...,/tensor5/data1/jonofrey/Liver/Paris/registrati...,/tensor5/data1/mgross/Liver/Paris/00004/00004_...,/tensor5/data1/jonofrey/Liver/Paris/registrati...,/tensor5/data1/mgross/Liver/Paris/man_liver_se...
3,6,/tensor5/data1/jonofrey/Liver/Paris/registrati...,/tensor5/data1/jonofrey/Liver/Paris/registrati...,/tensor5/data1/mgross/Liver/Paris/00006/00006_...,/tensor5/data1/jonofrey/Liver/Paris/registrati...,/tensor5/data1/mgross/Liver/Paris/man_liver_se...
4,10,/tensor5/data1/jonofrey/Liver/Paris/registrati...,/tensor5/data1/jonofrey/Liver/Paris/registrati...,/tensor5/data1/mgross/Liver/Paris/00010/00010_...,/tensor5/data1/jonofrey/Liver/Paris/registrati...,/tensor5/data1/mgross/Liver/Paris/man_liver_se...
5,11,/tensor5/data1/jonofrey/Liver/Paris/registrati...,/tensor5/data1/jonofrey/Liver/Paris/registrati...,/tensor5/data1/mgross/Liver/Paris/00011/00011_...,/tensor5/data1/jonofrey/Liver/Paris/registrati...,/tensor5/data1/mgross/Liver/Paris/man_liver_se...
6,12,/tensor5/data1/jonofrey/Liver/Paris/registrati...,/tensor5/data1/jonofrey/Liver/Paris/registrati...,/tensor5/data1/mgross/Liver/Paris/00012/00012_...,/tensor5/data1/jonofrey/Liver/Paris/registrati...,/tensor5/data1/mgross/Liver/Paris/man_liver_se...
7,17,/tensor5/data1/jonofrey/Liver/Paris/registrati...,/tensor5/data1/jonofrey/Liver/Paris/registrati...,/tensor5/data1/mgross/Liver/Paris/00017/00017_...,/tensor5/data1/jonofrey/Liver/Paris/registrati...,/tensor5/data1/mgross/Liver/Paris/man_liver_se...
8,18,/tensor5/data1/jonofrey/Liver/Paris/registrati...,/tensor5/data1/jonofrey/Liver/Paris/registrati...,/tensor5/data1/mgross/Liver/Paris/00018/00018_...,/tensor5/data1/jonofrey/Liver/Paris/registrati...,/tensor5/data1/mgross/Liver/Paris/man_liver_se...
9,19,/tensor5/data1/jonofrey/Liver/Paris/registrati...,/tensor5/data1/jonofrey/Liver/Paris/registrati...,/tensor5/data1/mgross/Liver/Paris/00019/00019_...,/tensor5/data1/jonofrey/Liver/Paris/registrati...,/tensor5/data1/mgross/Liver/Paris/man_liver_se...


In [6]:
#df_pre = pd.read_csv('../ExternalTestingResults/Results-PRE.csv')
df_art = pd.read_csv("../ExternalTestingResults/Results-ART-Net.csv")
df_pv = pd.read_csv("../ExternalTestingResults/Results-PV-Net.csv")
df_del = pd.read_csv("../ExternalTestingResults/Results-DEL-Net.csv")


In [7]:
for i in [df_art]:
    for idx, row in  i.iterrows():
            i.loc[idx, 'ACCESSION_ID'] = str(row['AUTOMATED-LIVER-SEGM'].split('/')[-1].split('_')[-2].split('tensor(')[-1].split(')')[0])


In [8]:
for i in [df_pv]:
    for idx, row in  i.iterrows():
            i.loc[idx, 'ACCESSION_ID'] = str(row['AUTOMATED-LIVER-SEGM'].split('/')[-1].split('_')[-2].split('tensor(')[-1].split(')')[0])


In [9]:
for i in [df_del]:
    for idx, row in  i.iterrows():
            i.loc[idx, 'ACCESSION_ID'] = str(row['AUTOMATED-LIVER-SEGM'].split('/')[-1].split('_')[-2].split('tensor(')[-1].split(')')[0])


In [10]:
#df_pre.rename(columns={'AUTOMATED-LIVER-SEGM' : 'PRE-SEGM'}, inplace = True)
df_art.rename(columns={'AUTOMATED-LIVER-SEGM' : 'ART-SEGM'}, inplace = True)
df_pv.rename(columns={'AUTOMATED-LIVER-SEGM' : 'PV-SEGM'}, inplace = True)
df_del.rename(columns={'AUTOMATED-LIVER-SEGM' : 'DEL-SEGM'}, inplace = True)


In [11]:
df = pd.merge(df_art, df_pv, on = 'ACCESSION_ID')
df = pd.merge(df, df_del, on = 'ACCESSION_ID')
df = pd.merge(df, df_gt, on = 'ACCESSION_ID')

# LATE FUSION

In [12]:
import os, numpy as np, pandas as pd, nibabel as nib
from nibabel.processing import resample_from_to

def load_canonical_bool(path):
    img = nib.load(str(path))
    img = nib.as_closest_canonical(img)
    arr = img.get_fdata(dtype=np.float32)
    if arr.ndim > 3: arr = arr[..., 0]
    return (arr > 0), img  # bool mask + Nifti1Image (canonical)

def nn_resample_to(src_mask_bool, src_img, ref_img):
    # wrap bool as uint8 image on src grid
    src_u8 = nib.Nifti1Image(src_mask_bool.astype(np.uint8), src_img.affine, src_img.header)
    rs = resample_from_to(src_u8, (ref_img.shape, ref_img.affine), order=0)  # nearest
    rs_arr = np.asarray(rs.dataobj)
    return (rs_arr > 0.5)  # bool on ref grid

def dice_bool(a, b):
    a = a.astype(bool); b = b.astype(bool)
    inter = np.logical_and(a, b).sum()
    s = a.sum() + b.sum()
    return 1.0 if s == 0 else (2.0 * inter) / s

def fuse_on_gt_and_score(row, out_dir=None, save_name=None):
    gt_path   = row["GT-SEGM"]
    art_path  = row["ART-SEGM"]
    pv_path   = row["PV-SEGM"]
    del_path  = row["DEL-SEGM"]

    # 1) load GT as reference grid
    gt_mask, gt_img = load_canonical_bool(gt_path)

    # 2) load preds and resample each to GT grid (nearest, stays binary)
    art_mask, art_img = load_canonical_bool(art_path)
    pv_mask , pv_img  = load_canonical_bool(pv_path)
    del_mask, del_img = load_canonical_bool(del_path)

    art_rs = nn_resample_to(art_mask, art_img, gt_img)
    pv_rs  = nn_resample_to(pv_mask , pv_img , gt_img)
    del_rs = nn_resample_to(del_mask, del_img, gt_img)

    # 3) majority vote (binary)
    votes = art_rs.astype(np.uint8) + pv_rs.astype(np.uint8) + del_rs.astype(np.uint8)
    fused = (votes >= 2)

    # 4) DSC vs GT (both on GT grid)
    dsc = dice_bool(fused, gt_mask)

    # Optional: save fused mask on GT grid
    if out_dir is not None:
        os.makedirs(out_dir, exist_ok=True)
        name = save_name or (str(row["ACCESSION_ID"]) if "ACCESSION_ID" in row else "fused")
        out_path = os.path.join(out_dir, f"{name}_fused_majority_gtgrid.nii.gz")
        hdr = gt_img.header.copy(); hdr.set_data_dtype(np.uint8)
        nib.save(nib.Nifti1Image(fused.astype(np.uint8), gt_img.affine, hdr), out_path)
        return dsc, out_path

    return dsc, None

In [13]:
OUT_DIR = "/tensor5/data1/mgross/Liver/studies/Contrast-Nets/LateFusion3-ART-PV-DEL/PARIS "


dsc_vals, fused_paths = [], []
for idx, row in df.iterrows():
    try:
        dsc, p = fuse_on_gt_and_score(row, out_dir=OUT_DIR, save_name=str(row.get("ACCESSION_ID", idx)))
    except Exception as e:
        print(f"[{idx}] error:", e)
        dsc, p = np.nan, None
    dsc_vals.append(dsc); fused_paths.append(p)
    

In [14]:
df["LATE-FUSION-SEGM"] = fused_paths
df["DSC_LATE_FUSION"]  = dsc_vals
print("Mean DSC:", np.nanmean(df["DSC_LATE_FUSION"]))

Mean DSC: 0.9610542038239391


In [15]:
df.to_csv("../ExternalTestingResults/PARIS-LATE_FUSION-ART-PV-DEL.csv")